# Full Text Reading 

> Basic Experiments with Pipelines to read full text papers as part of CUSO sprint on corpus reviews about transcriptomics and machine learning.    

In [1]:
import json

import os
import pandas as pd
from pprint import pprint
import textwrap

import sqlalchemy as db
from sqlalchemy.engine import create_engine
import json

import os
import pandas as pd
from pprint import pprint
import textwrap



# NXML Analysis

In [27]:
engine = create_engine(
    'databricks://token:'+os.environ['DB_TOKEN']+ 
    '@czi-shared-infra-czi-sci-general-prod-databricks.cloud.databricks.com'+
    '?http_path=/sql/1.0/warehouses/1c4df94f2f1a6305')

doi = '10.1101/2022.04.12.488077'

sql1 = '''
SELECT DISTINCT p.id, p.doi, p.pubYear, p.title, p.abstract, ft.text
FROM scipubstore.ingestion.papers as p  
    LEFT JOIN scipubstore.europe_pmc.articles as ft on (ft.pmc_id=p.pmcid) 
WHERE p.doi = '{}' 
'''.format(doi)

sql = '''
SELECT DISTINCT full_text_url
FROM scipubstore.ingestion.papers as p  
WHERE p.doi = '{}' 
'''.format(doi)

with engine.connect() as con:
    stmt = db.text(sql)
    rs = con.execute(stmt)
    df = pd.DataFrame(rs.fetchall(), columns=rs.keys())
df

#metadata = db.MetaData()
#metadata.reflect(bind=engine)
#tbls = metadata.tables
#print(json.dumps([str(t) for t in metadata.tables], indent=4))

#logs = Table("corpus", MetaData(bind=engine), autoload=True)
with engine.connect() as con:
    stmt = db.text(sql)
    rs = con.execute(stmt)
    df = pd.DataFrame(rs.fetchall(), columns=rs.keys())

url = df['full_text_url'][0]
#print(select([func.count("*")], from_obj=logs).scalar())
print(url)

import requests
r = requests.get(url)
print(r.status_code)
print(r.headers['content-type'])
#print(r.encoding)
#print(r.json())

xml = r.text
xml

https://www.biorxiv.org/content/early/2022/04/13/2022.04.12.488077.source.xml
200
application/xml


'<article article-type="article" specific-use="production" xml:lang="en" xmlns:hw="org.highwire.hpp" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:ali="http://www.niso.org/schemas/ali/1.0/" xmlns:ref="http://schema.highwire.org/Reference" xmlns:hwp="http://schema.highwire.org/Journal" xmlns:l="http://schema.highwire.org/Linking" xmlns:r="http://schema.highwire.org/Revision" xmlns:x="http://www.w3.org/1999/xhtml" xmlns:app="http://www.w3.org/2007/app" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:nlm="http://schema.highwire.org/NLM/Journal" xmlns:a="http://www.w3.org/2005/Atom" xmlns:c="http://schema.highwire.org/Compound" xmlns:hpp="http://schema.highwire.org/Publishing"><front><journal-meta><journal-id journal-id-type="hwp">biorxiv</journal-id><journal-id journal-id-type="publisher-id">BIORXIV</journal-id><journal-title>bioRxiv</journal-title><abbrev-journal-title abbrev-type="publisher">bioRxiv</abbrev-journal-title><pub

In [28]:
# Translate XML to stardard format 
from alhazen.utils.jats_text_extractor import NxmlDoc
doc = NxmlDoc(doi, xml)

In [9]:
df = doc.build_document_dataframe()
df

,PMID,PARAGRAPH_ID,TAG,TAG_TREE,OFFSET,LENGTH,FIG_REF,PLAIN_TEXT
0,10.1101/2022.04.12.488077,0,article-title,,228,90,,Convolutional networks for supervised mining o...
1,10.1101/2022.04.12.488077,1,abstract,,1885,1216,,AbstractCryo-electron tomograms capture a weal...
2,10.1101/2022.04.12.488077,2,title,Introduction,3270,12,,Introduction
3,10.1101/2022.04.12.488077,3,p,Introduction,3282,1006,,Cryo-Electron Tomography (cryo-ET) produces th...
4,10.1101/2022.04.12.488077,4,p,Introduction,4288,2166,,A range of available semi-automated methods fo...
...,...,...,...,...,...,...,...,...
124,10.1101/2022.04.12.488077,124,p,Discussion,61710,1434,,Subtomogram averaging of the annotated particl...
125,10.1101/2022.04.12.488077,125,p,Discussion,63144,1503,,The analysis of DeePiCt's performance confirme...
126,10.1101/2022.04.12.488077,126,p,Discussion,64647,1598,,Our workflow allows easy adaptation to the seg...
127,10.1101/2022.04.12.488077,127,title,supplementary-material,66245,22,,Supporting information


In [10]:
df[df.PLAIN_TEXT.str.contains('annotation')]

,PMID,PARAGRAPH_ID,TAG,TAG_TREE,OFFSET,LENGTH,FIG_REF,PLAIN_TEXT
8,10.1101/2022.04.12.488077,8,p,Results,7384,1500,,DeePiCt is based on deep learning technologies...
19,10.1101/2022.04.12.488077,19,p,Results,13713,600,figE2,Isosurface view of organelle ground truth anno...
20,10.1101/2022.04.12.488077,20,title,Results,14313,64,,Generation of comprehensive ground-truth annot...
21,10.1101/2022.04.12.488077,21,p,Results,14377,343,,The ultimate requirement for developing and be...
22,10.1101/2022.04.12.488077,22,p,Results,14720,1169,,"In cryo-ET data, the complete annotation of a ..."
24,10.1101/2022.04.12.488077,24,title,Results,15935,74,fig2,Iterative comprehensive annotation of ground t...
25,10.1101/2022.04.12.488077,25,p,Results,16009,1401,fig2,The three columns (left to right) show: the an...
26,10.1101/2022.04.12.488077,26,p,Results,17410,450,,The resulting ground truth RIBO annotation in ...
29,10.1101/2022.04.12.488077,29,p,Results,18006,1162,figE3,a-b. Contributions of the steps applied during...
30,10.1101/2022.04.12.488077,30,p,Results,19168,705,,The final ground truth annotation of FAS in th...


# Python-based PDF Text Extraction 

In [1]:
# papers about foundational large systems biology ML models  
d = '/Users/gburns/Library/CloudStorage/GoogleDrive-gully.burns@chanzuckerberg.com/Shared drives/' \
    + 'Science/SciTech Strategy/5_Learning Sprint/4_Challenges Use Cases Outcomes/reading/foundational_models/scRNAseq models/'
f = 'yang-2022-4-852-scBERT.pdf'

d2 = '/Users/gburns/Library/CloudStorage/GoogleDrive-gully.burns@chanzuckerberg.com/Shared drives/' \
    + 'Science/SciTech Strategy/5_Learning Sprint/4_Challenges Use Cases Outcomes/reading//background_applications/'
f2 = 'yang-2023-8-115-alphafold2apps.pdf'


In [6]:
# papers about CryoET paper  
import os
d = '/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/'

# list papers in directory
papers = [f for f in os.listdir(d) if f.endswith('.pdf')]
print(papers)
p = 'de_teresa-2022-10.1101-2022.04.12.488077.pdf'
p2 = 'de_teresa-tueba-2023-20-284.pdf'

['rangan-2023-10.1101-2023.08.18.553799.pdf', 'ke-2020-10.1101-2020.06.27.174979.pdf', 'barad-2022-10.1101-2022.01.23.477440.pdf', 'xue-2022-610-205.pdf', 'chen-2022-10.1101-2022.09.28.510016.pdf', 'de_teresa-2022-10.1101-2022.04.12.488077.pdf', 'de_teresa-tueba-2023-20-284.pdf', 'chen-2023-30-360.pdf', 'tegunov-2020-10.1101-2020.06.05136341.pdf', 'tegunov-2021-18-186.pdf', 'barad-2023-222-e202204093.pdf', 'ke-2020-588-498.pdf', 'hoek-2022-377-543.pdf']


In [7]:
from langchain.document_loaders.pdf import BasePDFLoader
from langchain.document_loaders.base import BaseBlobParser
from typing import Optional, List, Iterator, Mapping, Any, Dict
from langchain.docstore.document import Document
from langchain.document_loaders.blob_loaders import Blob

import re

import dataclasses

@dataclasses.dataclass
class PyMuPDFBlock:
    """A block of text occuring in a PDF file."""

    page: int

    x0:float
    x1:float
    width:float

    y0:float
    y1:float
    height:float
    
    nlines:int
    font_size_proxy:float

    text:str
    t:str

    def __init__(self, p:int, x0:float, y0:float, x1:float, y1:float, text:str):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.page = p
        
        self.text = text
        self.width = x1 - x0
        self.height = y1 - y0
        self.nlines = text.count('\n')
        if self.nlines == 0:
            self.nlines = 1 
        self.font_size_proxy = (self.height / self.nlines)
        self.t = re.sub('\-\n','',self.text)
        self.t = re.sub('\n',' ',self.text)
        self.t = re.sub('\s+',' ',self.text)
        
    def __repr__(self):
        return f"PyMuPDFBlock(page={self.page}, x0={self.x0}, x1={self.x1}, y0={self.y0}, y1={self.y1}, text={self.text})"
    
    def __str__(self): 
        return f"PyMuPDFBlock(page={self.page}, x0={self.x0}, x1={self.x1}, y0={self.y0}, y1={self.y1}, text={self.text})"
    
    def __eq__(self, other):
        if self.page == other.page and self.x0 == other.x0 and self.x1 == other.x1 and self.y0 == other.y0 and self.y1 == other.y1:
            return True
        else:
            return False

class PyMuPDFBlockLoader(BasePDFLoader):
    """Load `PDF` files using `PyMuPDF`."""

    def __init__(self, file_path: str) -> None:
        """Initialize with a file path."""
        try:
            import fitz  # noqa:F401
        except ImportError:
            raise ImportError(
                "`PyMuPDF` package not found, please install it with "
                "`pip install pymupdf`"
            )

        super().__init__(file_path)

    def load(self, **kwargs: Optional[Any]) -> List[Document]:
        """Load file."""

        parser = PyMuPDFBlockParser(text_kwargs=kwargs)
        blob = Blob.from_path(self.file_path)
        blocks = parser.parse(blob)
    
        return blocks
    
class PyMuPDFBlockParser(BaseBlobParser):
    """Parse `PDF` using `PyMuPDF`."""

    def __init__(self, text_kwargs: Optional[Mapping[str, Any]] = None) -> None:
        """Initialize the parser.

        Args:
            text_kwargs: Keyword arguments to pass to ``fitz.Page.get_text()``.
        """
        self.text_kwargs = text_kwargs or {}

    def lazy_parse(self, blob: Blob) -> Iterator[Document]:
        """Lazily parse the blob."""
        import fitz

        with blob.as_bytes_io() as file_path:

            doc = fitz.open(file_path)  # open document

            blocks = []
            text_dict = {}
            
            for page in doc:
                for (x0, y0, x1, y1, text, bn, bt) in page.get_textpage().extractBLOCKS():
                    b = PyMuPDFBlock(page.number, x0, y0, x1, y1, text)
                    blocks.append(b)
                    if text_dict.get(text):
                        text_dict[text].append(b)
                    else:
                        text_dict[text] = [b]

            # remove blocks that are repeated
            #for t in text_dict.keys():
            #    if len(text_dict[t]) > 1:
            #        for b in text_dict[t]:
            #            blocks.remove(b)
            
            # remove blocks that are urls on their own
            # remove any blocks that are one word long
            #to_remove = []
            #for b in blocks:
            #    m1 = re.search('^\s*http://[A-Za-z1-9\.\/\-\_]+\s*$', b.text)
            #    m2 = re.search('^\s*\S+\s*$', b.text)
            #    if m1 or m2:
            #       to_remove.append(b)

            for b in blocks:
                yield Document(
                    page_content=b.text,
                    metadata=dict({
                        "x0": b.x0, "x1": b.x1, 
                        "y0": y0, "y1": y1, 
                        "width": b.width, "height": b.height,
                        "nlines": b.nlines,
                        "fs": b.font_size_proxy,
                        "file_path": blob.source,
                        "page": page.number,
                        "total_pages": len(doc),
                    }))



loader = PyMuPDFBlockLoader(d+p)
blocks = loader.load()
blocks

[Document(page_content='Convolutional networks for supervised mining of molecular patterns within \ncellular context \n \n', metadata={'x0': 72.0, 'x1': 543.6431884765625, 'y0': 53.400001525878906, 'y1': 84.70805358886719, 'width': 471.6431884765625, 'height': 58.50422668457031, 'nlines': 3, 'fs': 19.50140889485677, 'file_path': '/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/de_teresa-2022-10.1101-2022.04.12.488077.pdf', 'page': 65, 'total_pages': 66}),
 Document(page_content='Irene de Teresa1,2*, Sara K. Goetz1,3*, Alexander Mattausch1,4, Frosina Stojanovska1,3,8, Christian E. \nZimmerli1,5, Mauricio Toro-Nahuelpan1,6, Dorothy W.C. Cheng1,3,7, Fergus Tollervey1,3, Constantin Pape7,9, \nMartin Beck1,5, Anna Kreshuk7, Julia Mahamid1,7+ and Judith Zaugg1,8+. \n \n', metadata={'x0': 72.0, 'x1': 542.76025390625, 'y0': 53.400001525878906, 'y1': 84.70805358886719, 'width': 470.76025390625, 'height': 56.59019470214844, 'nlines': 4, 'fs': 14.14754867553711, 'file_

In [8]:
l2 = PyMuPDFBlockLoader(d2+f2)
blocks = l2.load()

blocks


[Document(page_content='Convolutional networks for supervised mining of molecular patterns within \ncellular context \n \n', metadata={'x0': 72.0, 'x1': 543.6431884765625, 'y0': 53.400001525878906, 'y1': 84.70805358886719, 'width': 471.6431884765625, 'height': 58.50422668457031, 'nlines': 3, 'fs': 19.50140889485677, 'file_path': '/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/de_teresa-2022-10.1101-2022.04.12.488077.pdf', 'page': 65, 'total_pages': 66}),
 Document(page_content='Irene de Teresa1,2*, Sara K. Goetz1,3*, Alexander Mattausch1,4, Frosina Stojanovska1,3,8, Christian E. \nZimmerli1,5, Mauricio Toro-Nahuelpan1,6, Dorothy W.C. Cheng1,3,7, Fergus Tollervey1,3, Constantin Pape7,9, \nMartin Beck1,5, Anna Kreshuk7, Julia Mahamid1,7+ and Judith Zaugg1,8+. \n \n', metadata={'x0': 72.0, 'x1': 542.76025390625, 'y0': 53.400001525878906, 'y1': 84.70805358886719, 'width': 470.76025390625, 'height': 56.59019470214844, 'nlines': 4, 'fs': 14.14754867553711, 'file_

# Starting analysis with Langchain

In [1]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

n_gpu_layers = 1  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/Users/gburns/Documents/Coding/ChatGPT_etc/LLMs/llama-2-70b-chat.Q5_K_M.gguf",
    n_ctx=4096,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True, # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /Users/gburns/Documents/Coding/ChatGPT_etc/LLMs/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_outp

In [70]:
from __future__ import annotations

from abc import abstractmethod
from typing import List
import re

from langchain.schema import BaseOutputParser, OutputParserException

import json
from json import JSONDecodeError
from typing import Any, List

class JsonEnclosedByTextOutputParser(BaseOutputParser[Any]):
    """Parse the output of an LLM call to a JSON object."""

    def parse(self, text: str) -> Any:
        text = text.strip()
        m = re.search('.*([\[\{](.|\n)*[\}\]]).*', text, flags=re.M)
        if m:
            text1 = m.group(1)
            try:
                return json.loads(text1)
            except JSONDecodeError as e:
                return None
                #raise OutputParserException(f"Invalid json output: {text1} derived from {text}") from e
        else: 
            raise OutputParserException(f"Could not find json-formatted data in: {text}")

    @property
    def _type(self) -> str:
        return "json_enclosed_by_text_output_parser"
    

In [71]:
from langchain import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = JsonEnclosedByTextOutputParser()

In [18]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 30)
df.replace(r'\n',  ' ', inplace=True) 
df['PLAIN_TEXT'] = df['PLAIN_TEXT'].str.wrap(50)
df_p = df[df.TAG=='p']
df_p


,PMID,PARAGRAPH_ID,TAG,TAG_TREE,OFFSET,LENGTH,FIG_REF,PLAIN_TEXT
3,10.1101/2022.04.12.488077,3,p,Introduction,3282,1006,,Cryo-Electron Tomography (...
4,10.1101/2022.04.12.488077,4,p,Introduction,4288,2166,,A range of available semi-...
5,10.1101/2022.04.12.488077,5,p,Introduction,6454,825,,Here we present DeePiCt (D...
8,10.1101/2022.04.12.488077,8,p,Results,7384,1500,,DeePiCt is based on deep l...
11,10.1101/2022.04.12.488077,11,p,Results,9081,1328,fig1,a. The CNN U-Net architect...
...,...,...,...,...,...,...,...,...
122,10.1101/2022.04.12.488077,122,p,Discussion,60254,921,,Our DeePiCt workflow facil...
123,10.1101/2022.04.12.488077,123,p,Discussion,61175,535,,"Here, we provide the first..."
124,10.1101/2022.04.12.488077,124,p,Discussion,61710,1434,,Subtomogram averaging of t...
125,10.1101/2022.04.12.488077,125,p,Discussion,63144,1503,,The analysis of DeePiCt's ...


In [15]:
t3 = """[INST]
<<SYS>>
Your job is to carefully read an excerpt from a scientific research article and perform the task described below. 
The text of the excerpt will be delimited with triple backticks. 

Be precise in your answers. 
<</SYS>>

1. Split the text of the excerpt into separate sentences.

2. Identify if there are any sentences in the excerpt that describe the process of annotating a dataset obtained using electron microsopy. 
If there no such sentences, return an empty list.

3. If there are any sentences that describe how annotations were obtained, write a short summary of the methods used. 
If there no such sentences, respond with "No methods described."

In addition, explain your reasoning briefly in one or two sentences.

excerpt: '''{excerpt}'''
[INST]
"""
p3 = PromptTemplate(
    template=t3,
    input_variables=["excerpt"],
)

NameError: name 'PromptTemplate' is not defined

In [11]:
len(df[df.PLAIN_TEXT.str.contains('[Aa]nnotation')])

22

In [19]:
#for i, row in df[df.PLAIN_TEXT.str.contains('[Aa]nnotation')].iterrows():
#    prompt = p3.format(excerpt=row.PLAIN_TEXT)
#    output = llm(prompt)
print('clear out')


clear out


In [20]:
# check
r = df[df.ID_PAPER=="28945706"]
a = r.ABSTRACT.values[0]
t = r.TITLE.values[0]
prompt = p2.format(title=t, abstract=a)
print(prompt)
output = llm(p3.format(title=t, abstract=a))
#m = output_parser.parse(output)

print('\n\n\n'+output)

AttributeError: 'DataFrame' object has no attribute 'ID_PAPER'

In [11]:

os.environ["WEAVIATE_URL"] = 'http://localhost:8080/'

import weaviate
from langchain.vectorstores import Weaviate
from langchain.embeddings.llamacpp import LlamaCppEmbeddings

client = weaviate.Client(url=os.environ["WEAVIATE_URL"])
docs = [
    Document(page_content="foo", metadata={"id": 1}),
]
embeddings = LlamaCppEmbeddings(model_path="/Users/gburns/Documents/Coding/ChatGPT_etc/LLMs/llama-2-70b-chat.Q5_K_M.gguf")

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /Users/gburns/Documents/Coding/ChatGPT_etc/LLMs/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_outp

In [ ]:
client = weaviate.Client(os.environ["WEAVIATE_URL"])

w = Weaviate.from_documents(blocks, embeddings, weaviate_url=os.environ["WEAVIATE_URL"], by_text=False, index_name="MyIndex")
w.similarity_search_by_text(query="annotations")

In [10]:
db.similarity_search_by_text(query="annotations")

AttributeError: module 'sqlalchemy' has no attribute 'similarity_search_by_text'